# Solver Parameters Tutorial
For instructions on how to run these tutorial notebooks, please see the [index](./index.ipynb).


## Setting solver parameters
Many solvers allow the users to adjust the parameters. When calling `Solve()` function, Drake will use the default parameters for the solver (iterations, optimality tolerance, etc). You could modify these parameters in two ways, by either calling `MathematicalProgram::SetSolverOption`, or pass a `SolverOptions` argument to the `Solve()` function.

### Calling MathematicalProgram::SetSolverOption
By calling `MathematicalProgram::SetSolverOption(solver_id, option_name, option_value)`, you can set a parameter for a specific solver (with the matching `solver_id`). The `option_name` is specific to that solver (for example, [here](https://coin-or.github.io/Ipopt/OPTIONS.html) is a list of IPOPT parameters). Note that `MathematicalProgram` object will store this solver parameter, and this parameter will be applied in the `Solve()` call, if that specific solver (with the matching `solver_id`) is invoked.

In the following code snippet, we show an example of setting the options of IPOPT.

In [5]:
from pydrake.solvers import MathematicalProgram, SolverOptions, Solve
from pydrake.solvers import IpoptSolver
import numpy as np
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddCost(x[0]**2 + x[1] ** 2)
prog.AddConstraint(x[0] + x[1] == 1)

# Set the maximum iteration for IPOPT to be 1.
# max_iter is a parameter of IPOPT solver, explained in
# https://www.coin-or.org/Ipopt/documentation/node42.html
prog.SetSolverOption(IpoptSolver().solver_id(), "max_iter", 1)
solver = IpoptSolver()
result = solver.Solve(prog, np.array([10, 1]), None)
# With fewer maximum iteration, IPOPT hasn't converged to optimality yet (The true optimal is [0.5, 0.5])
print("Success? ", result.is_success())
print(result.get_solution_result())
print("IPOPT x*= ", result.GetSolution(x))

Success?  False
SolutionResult.kIterationLimit
IPOPT x*=  [-4.  5.]


Also note that setting the parameter of a solver **doesn't** mean that `result = Solve(prog)` will invoke that solver. The invoked solver is determined by Drake, to choose whichever solver it thinks most appropriate.

In the following snippet, although we set the solver options for IPOPT, Drake chooses another solver (which can solve this particular problem in the closed form.)

In [2]:
prog.SetSolverOption(IpoptSolver().solver_id(), "max_iter", 1)
result = Solve(prog)
print(result.get_solver_id().name())

EqConstrainedQP


### Passing a SolverOptions to Solve function
Another way of setting the solver options is to pass in a `SolverOptions` object as an argument to `Solve` function. `MathematicalProgram` will *not* store this `SolverOptions` object.

In the following example, in the first `Solve` call, it uses the `SolverOptions` object to set the parameter for IPOPT; in the second `Solve` call, it uses the default IPOPT parameters, hence we get different results from two `Solve` calls.

In [6]:
import pydrake
dir(pydrake.solvers.SnoptSolver)

['AddBilinearProductMcCormickEnvelopeSos2',
 'AddLogarithmicSos1Constraint',
 'AddLogarithmicSos2Constraint',
 'AddSos2Constraint',
 'AugmentedLagrangianNonsmooth',
 'AugmentedLagrangianSmooth',
 'Binding',
 'Binding𝓣BoundingBoxConstraint𝓤',
 'Binding𝓣Constraint𝓤',
 'Binding𝓣Cost𝓤',
 'Binding𝓣EvaluatorBase𝓤',
 'Binding𝓣ExponentialConeConstraint𝓤',
 'Binding𝓣ExpressionConstraint𝓤',
 'Binding𝓣ExpressionCost𝓤',
 'Binding𝓣L1NormCost𝓤',
 'Binding𝓣L2NormCost𝓤',
 'Binding𝓣LInfNormCost𝓤',
 'Binding𝓣LinearComplementarityConstraint𝓤',
 'Binding𝓣LinearConstraint𝓤',
 'Binding𝓣LinearCost𝓤',
 'Binding𝓣LinearEqualityConstraint𝓤',
 'Binding𝓣LinearMatrixInequalityConstraint𝓤',
 'Binding𝓣LorentzConeConstraint𝓤',
 'Binding𝓣MinimumValueConstraint𝓤',
 'Binding𝓣MinimumValueLowerBoundConstraint𝓤',
 'Binding𝓣MinimumValueUpperBoundConstraint𝓤',
 'Binding𝓣PerspectiveQuadraticCost𝓤',
 'Binding𝓣PositiveSemidefiniteConstraint𝓤',
 'Binding𝓣QuadraticConstraint𝓤',
 'Binding𝓣QuadraticCost𝓤',
 'Binding𝓣RotatedLorentzCo

In [3]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddCost(x[0]**2 + x[1] ** 2)
prog.AddConstraint(x[0] + x[1] == 1)

solver_options = SolverOptions()
solver_options.SetOption(IpoptSolver().solver_id(), "max_iter", 1)
solver = IpoptSolver()

# Call Solve with solver_options, IPOPT will use `max_iter` = 1
result = solver.Solve(prog, np.array([10, 1]), solver_options)
print("Success? ", result.is_success())
print(result.get_solution_result())
# Call Solve without solver_options, IPOPT will use the default options.
result = solver.Solve(prog, np.array([10, 1]), None)
print("Success? ", result.is_success())
print(result.get_solution_result())

Success?  False
SolutionResult.kIterationLimit
Success?  True
SolutionResult.kSolutionFound


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2b4fc509-aef2-417d-a40d-6071dfed9199' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>